In [1]:
import pandas as pd
import zipfile as zp
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans 
from sklearn.preprocessing import  scale

In [2]:
!wget -cq https://www.dropbox.com/s/1kasx9z1g4qdxjy/Solar%20Farm%20PMs%20and%20WS%20Data.zip
files = zp.ZipFile('/content/Solar Farm PMs and WS Data.zip')

In [3]:
def get_weather_data(df ):
    return df[ df.PME_SourceName == 'WP_SF_MVPS4.WS1'].pivot_table(  index = 'TimestampLocal' , columns='PME_MeasurementName' , values =  'PME_Value' )

In [4]:
def get_active_power(df, measurement_needed):
    dat =  df[  df.PME_MeasurementName == measurement_needed ].pivot_table( index = 'TimestampLocal' , columns='PME_SourceName' , values =  'PME_Value' )
    return dat[['WP_SF_MVPS1.PM1' , 'WP_SF_MVPS2.PM1' , 'WP_SF_MVPS3.PM1' , 'WP_SF_MVPS4.PM1' ]]
    #return dat

In [5]:
dfs = []

for index,each in enumerate(files.namelist()):
  if( (index % 2 == 0) & (index!=0)  ):
    print(index,each)
    f = files.open(each)
    df = pd.read_csv(f)
    weather_data =  get_weather_data(df)
    active_power =   get_active_power(df, 'Active Power')
    merged = pd.merge(active_power,weather_data,how = 'left' , on ='TimestampLocal')
    merged.reset_index(inplace=True)
    merged['TimestampLocal'] =  pd.to_datetime( merged['TimestampLocal'] , format = '%d/%m/%Y %I:%M:%S.%f %p' )  #merged['TimestampLocal'].astype('datetime64[ns]')
    merged.set_index('TimestampLocal' , inplace=True)
    dfs.append(merged)



2 Solar Farm PMs and WS Data/Digital Twin PME Data Export - Solar Farm PMs and WS.PME_FEB2021.csv
4 Solar Farm PMs and WS Data/Digital Twin PME Data Export - Solar Farm PMs and WS.PME_DataLog.MAR2021.csv
6 Solar Farm PMs and WS Data/Digital Twin PME Data Export - Solar Farm PMs and WS.PME_DataLog.JUL2021.csv
8 Solar Farm PMs and WS Data/Digital Twin PME Data Export - Solar Farm PMs and WS.PME_DataLog.JUN2021.csv
10 Solar Farm PMs and WS Data/Digital Twin PME Data Export - Solar Farm PMs and WS.PME_DataLog.AUG2020.csv
12 Solar Farm PMs and WS Data/Digital Twin PME Data Export - Solar Farm PMs and WS.PME_DataLog.MAY2021.csv
14 Solar Farm PMs and WS Data/Digital Twin PME Data Export - Solar Farm PMs and WS.PME_DataLog.JAN2021.csv
16 Solar Farm PMs and WS Data/Digital Twin PME Data Export - Solar Farm PMs and WS.PME_DataLog.OCT2020.csv
18 Solar Farm PMs and WS Data/Digital Twin PME Data Export - Solar Farm PMs and WS.PME_DataLog.SEP2020.csv
20 Solar Farm PMs and WS Data/Digital Twin PME Da

In [6]:
main = pd.concat(dfs)
main.drop_duplicates( keep = 'first' , inplace = True )
main.isna().sum()
display(main.sort_index().head(5))
sorted = main.sort_index()


,WP_SF_MVPS1.PM1,WP_SF_MVPS2.PM1,WP_SF_MVPS3.PM1,WP_SF_MVPS4.PM1,AVG in-plane irradiance,Ambient Temperature Celsius,Back-of-Module Temperature (deg C),Back-of-Module Temperature 2 (deg C),Irradiance Global (W/m^2),Irradiance Plane-of-Array (W/m^2),Rainfall (mm),Weather Atmospheric Pressure,Weather Relative Humidity,Wind Direction (Hour Interval),Wind Speed
TimestampLocal,,,,,,,,,,,,,,,
2020-08-01 00:15:00,0.0,0.0,0.0,0.0,0.0,8.4,5.9,5.5,0.0,0.0,76.4,1013.0,81.0,307.0,1.0
2020-08-01 00:30:00,0.0,0.0,0.0,0.0,0.0,8.3,5.8,5.4,0.0,0.0,76.4,1013.0,82.0,331.0,3.0
2020-08-01 00:45:00,0.0,0.0,0.0,0.0,0.0,8.2,5.8,5.3,0.0,0.0,76.4,1013.0,82.0,337.0,2.0
2020-08-01 01:00:00,0.0,0.0,0.0,0.0,0.0,8.0,5.3,4.8,0.0,0.0,76.4,1013.0,82.0,315.0,3.0
2020-08-01 01:15:00,0.0,0.0,0.0,0.0,0.0,8.0,5.5,5.1,0.0,0.0,76.4,1013.0,82.0,329.0,3.0


## Low pass filter outlier detection

In [27]:
#All the columns in our data set with their index 
features_Df = pd.DataFrame( list(sorted.columns ) , columns=['Features'] ) 
features_Df

,Features
0,WP_SF_MVPS1.PM1
1,WP_SF_MVPS2.PM1
2,WP_SF_MVPS3.PM1
3,WP_SF_MVPS4.PM1
4,AVG in-plane irradiance
5,Ambient Temperature Celsius
6,Back-of-Module Temperature (deg C)
7,Back-of-Module Temperature 2 (deg C)
8,Irradiance Global (W/m^2)
9,Irradiance Plane-of-Array (W/m^2)


In [17]:
import pandas as pd 
import numpy as np



def find_outliers(df , w):
  '''
  This function will compute anamolies in pandas seriers 

  param df: Pandas series to find anamolies in 
  param w: window size for rolling function 

  The Global variable index_of_outliers will have list of all the outliers 
  '''
  TH = df.rolling(window = w  ).mean() +  ( 3 * df.rolling(window = w ).std() ) 
  TL = df.rolling(window = w  ).mean() -  ( 3 * df.rolling(window = w ).std() ) 
  concated  =  pd.concat( [df,TH,TL] , axis = 1)
  concated.columns = ['Power','TH','TL']
  
  concated.TH = concated.TH.shift( +1)
  concated.TL = concated.TL.shift( +1 )

  if len(concated[(concated.Power > concated.TH)| (concated.Power < concated.TL ) ]):
    check = list(concated[(concated.Power > concated.TH) | (concated.Power < concated.TL )].index)[0]
    index_of_outliers.append( check  )
    df.drop( check  , axis = 'index'  , inplace = True  )
    find_TH_TL( df , w)
  else:
    print('All outliers detected')
    return df



### Testing the function 

In [18]:
np.random.seed(123)
df = pd.Series(np.random.normal(size=200))

# Create a few outliers (3 of them, at index locations 10, 55, 80)
df[[13, 55, 80]] = 10.   
index_of_outliers = []
coopy = df.copy( deep=True  )
find_outliers(df , 12 )
print( 'outliers-Detected = '  , index_of_outliers ) 
fig = px.scatter(  x = coopy.index , y = coopy )
fig.add_scatter( x =  index_of_outliers ,  y =  coopy.loc[index_of_outliers] , mode='markers'  )
fig.show() 


All outliers detected
outliers-Detected =  [13, 46, 55, 80, 185, 195]


In [34]:
index_of_outliers = []
def preprocess_data( observations  , col_num  , w = 12  ):
  '''
  This function will help to select a feature of intrest to check for anamolies and plots outliers by calling the find_outliers function. 

  param observations: An integer value specifying the number of observations to select 
  param col_num: This int value indicated the feature to select, features avalable and their corresponding index are in the starting cell of this section.
  param w: window size that will be passed to the find_outliers
  '''
  index_of_outliers.clear()
  test_Data = sorted.iloc[ : observations  ,  col_num   ]
  test_Data = test_Data[test_Data > 0  ]
  copy_of_test_data = test_Data.copy( deep = True  )
  find_TH_TL(test_Data , w ) 
  fig = px.scatter(  x = copy_of_test_data.index , y = copy_of_test_data , title='Anamoly detection' , labels={
                     "x": "Time",
                     "y": features_Df.loc[col_num]['Features']
                     
                 }, )
  fig.add_scatter( x =  index_of_outliers ,  y =  copy_of_test_data.loc[index_of_outliers]  , mode = 'markers' , name='OutLiers' )
  fig.show()

In [35]:
#Irradiance 
preprocess_data(96  , 4 , w = 30)  #there are  96 observations per day 
#Wind Speed 
preprocess_data(96, 14 , w = 30)
#Weather Atmospheric Pressure
preprocess_data(96, 11 , w = 30)

All outliers detected


All outliers detected


All outliers detected
